## Load Required files

In [1]:
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
#------------------------Read in data ---------------------------
#SA Cummulative
df = pd.read_csv('./../data/external/SA_hourly_update.csv')

#Provincial cummulative
df_day=pd.read_csv('./../data/external/provincial_cumulative.csv')

# Commulative death per province 

df_death = pd.read_csv('./../data/external/provincial_death.csv')

#Update for the most recent 24 hours: SA Cases
df_day.head()

,Unnamed: 0,date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
0,0,05-03-2020,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
1,1,07-03-2020,20200307,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN
2,2,08-03-2020,20200308,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3,so
3,3,09-03-2020,20200309,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7,NaN
4,4,11-03-2020,20200311,0.0,0.0,5.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,13,NaN


In [3]:
df_day.columns

Index(['Unnamed: 0', 'date', 'YYYYMMDD', 'EC', 'FS', 'GP', 'KZN', 'LP', 'MP',
       'NC', 'NW', 'WC', 'UNKNOWN', 'total', 'source'],
      dtype='object')

In [ ]:
#Drop unwanted columns
# Provincial cases dataset
df_day = df_day.drop(['source','Unnamed: 0','YYYYMMDD'],axis = 1)
df_day = df_day.dropna()
# Provincial cases dataset
df_death = df_death.drop(['source','Unnamed: 0','YYYYMMDD'],axis = 1)
df_death = df_death.dropna()

In [4]:

#Commulative cases by province


fig_comm = go.Figure()
fig_comm.add_trace(
    go.Scatter(
        y=df_day['EC'],
        x=df_day.date,
        name= 'EC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['WC'],
        x=df_day.date,
        name= 'WC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['FS'],
        x=df_day.date,
        name= 'FS'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['GP'],
        x=df_day.date,
        name= 'GP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['KZN'],
        x=df_day.date,
        name= "KZN"
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NW'],
        x=df_day.date,
        name= 'NW'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['LP'],
        x=df_day.date,
        name= 'LP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['MP'],
        x=df_day.date,
        name= 'MP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NC'],
        x=df_day.date,
        name= 'NC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['UNKNOWN'],
        x=df_day.date,
        name='Unlocated'
    ))
# fig_comm.add_trace(
#     go.Scatter(
#         y=df_day['total'],
#         x=df_day.date,
#         name='Total Confirmed in SA'
#     ))

fig_comm.update_layout(hovermode='x',
                       title = 'Commulative confirmed cases by province',
                       xaxis=dict(
                         rangeslider=dict(
                             visible = True),
                           
       
                       
    ))


In [8]:
# df_day  =df.iloc[:, lambda df: [1, 2]]
import plotly.offline as py
import plotly.graph_objs as go
# py.iplot([{
#     'x': df_day.date,
#     'y': df_day[col],
#     'name': col
# }  for col in df_day.columns], )

In [14]:
df_day.head()

,date,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,05-03-2020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,07-03-2020,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,08-03-2020,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,09-03-2020,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7
4,11-03-2020,0.0,0.0,5.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,13


In [15]:


#Commulative deaths by province


fig = go.Figure()
fig_comm.add_trace(
    go.Scatter(
        y=df_death['EC'],
        x=df_death.date,
        name= 'EC'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['WC'],
        x=df_death.date,
        name= 'WC'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['FS'],
        x=df_death.date,
        name= 'FS'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['GP'],
        x=df_death.date,
        name= 'GP'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['KZN'],
        x=df_death.date,
        name= "KZN"
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['NW'],
        x=df_death.date,
        name= 'NW'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['LP'],
        x=df_death.date,
        name= 'LP'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['MP'],
        x=df_death.date,
        name= 'MP'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['NC'],
        x=df_death.date,
        name= 'NC'
    ))
fig.add_trace(
    go.Scatter(
        y=df_death['UNKNOWN'],
        x=df_death.date,
        name='Unlocated'
    ))
# fig.add_trace(
#     go.Scatter(
#         y=df_death['total'],
#         x=df_death.date,
#         name='Total Confirmed in SA'
#     ))

fig.update_layout(hovermode='x',
                       title = 'Commulative Death cases by province',
                       xaxis=dict(
                         rangeslider=dict(
                             visible = True),
                           
       
                       
    ))


In [18]:
#SA commulative data graph

fig = go.Figure()
fig.add_trace(go.Scatter(x = df.timestamp,y= df.active, name = 'Active Cases in SA'))
fig.update_layout(title = 'Commulative confirmed cases in SA as 17/03/2020')
fig.show()

In [19]:
df

,Unnamed: 0,cases,todayCases,deaths,todayDeaths,recovered,active,critical,timestamp
0,0,62,0,0,0,0,62,0,2020-03-17T21:00:00.200Z
1,1,62,0,0,0,0,62,0,2020-03-17T22:00:00.182Z
2,2,85,21,0,0,0,85,0,2020-03-17T23:00:00.171Z
3,3,85,21,0,0,0,85,0,2020-03-18T00:00:00.193Z
4,4,85,21,0,0,0,85,0,2020-03-18T01:00:00.205Z
...,...,...,...,...,...,...,...,...,...
1294,1294,10015,0,194,0,4173,5648,77,2020-05-11T09:00:00.177Z
1295,1295,10015,0,194,0,4173,5648,77,2020-05-11T10:00:00.267Z
1296,1296,10015,0,194,0,4173,5648,77,2020-05-11T11:00:00.220Z
1297,1297,10015,0,194,0,4173,5648,77,2020-05-11T12:00:00.280Z
